In [37]:
import os
import numpy as np
import sys, math, random

train_neg_data_path = '/Users/mac/Desktop/IMDB/aclImdb/train/neg'
train_pos_data_path = '/Users/mac/Desktop/IMDB/aclImdb/train/pos'

seed = 1
np.random.seed(1) 
random.seed(1)
reviews = []
labels = []
         
for file in os.listdir(train_neg_data_path):
    with open(f'{train_neg_data_path}/{file}') as f:
        rev = f.readline()
        for sign in ("<br />","/","\\",")","(",";",":","?","!","<",">","#","$",",","]","["):
            rev = rev.replace(sign, '')
            rev = rev.replace('.', ' ')
            rev = rev.replace("'", ' ')
        reviews.append(rev.lower())
    labels.append('negative\n')

for file in os.listdir(train_pos_data_path):
    with open(f'{train_pos_data_path}/{file}') as f:
        rev = f.readline()
        for sign in ("<br />","/","\\",")","(",";",":","?","!","<",">","#","$",",","]","["):
            rev = rev.replace(sign, '')
            rev = rev.replace('.', ' ') 
            rev = rev.replace("'", ' ') 
        reviews.append(rev.lower())
    labels.append('positive\n')
print('done')

done


In [38]:
from collections import Counter

tokens = list(map(lambda x:(x.split(' ')), reviews))
wordcnt = Counter()
for sent in tokens:
    for word in sent:
        wordcnt[word] -= 1   #счетчик - кортежи со словом и сколько раз встречается
vocab = list(set(map(lambda x:x[0], wordcnt.most_common())))

word2index = {}
for i, word in enumerate(vocab):
    word2index[word] = i
    
concatenated = list()
input_dataset = list()

for sent in tokens:
    sent_indices = list()
    
    for word in sent:
        try:
            sent_indices.append(word2index[word])
            concatenated.append(word2index[word])
        except:
            ""
    input_dataset.append(sent_indices)
concatenated = np.array(concatenated)    #все слова во всех обзорах - 3840053
print('donе')

donе


In [39]:

random.shuffle(input_dataset)

alpha, iterations = (0.05, 2)
hidden_size, window, negative = (50, 2, 5)

weight_0_1 = (np.random.rand(len(vocab), hidden_size) - 0.5) * 0.2
weight_1_2 = np.zeros((len(vocab), hidden_size))               

weight_0_1.shape, weight_1_2.shape

layer_2_target = np.zeros(negative+1)
layer_2_target[0] = 1
 
def similar(target='terrible'):
    target_index = word2index[target]
    scores = Counter()
    
    for word, index in word2index.items():
        raw_difference = weight_0_1[index] - (weight_0_1[target_index])
        squared_difference = raw_difference * raw_difference
        scores[word] = -math.sqrt(sum(squared_difference))
    return scores.most_common(10)

def sigmoid(x):
    return 1/(1 + np.exp(-x))

for rev_i, review in enumerate((input_dataset*iterations)):   #список увеличивается в iterations раз
    #print(rev_i,' ',review)
    for target_i in range(len(review)):
        
        #Слово из обзора + 5 случайных индексов:
        target_samples = [review[target_i]] + list(concatenated[(np.random.rand(negative)*len(concatenated)).astype('int').tolist()])
        #print(target_samples) [77810, 48283, 35507, 9606, 18169]        
        
        left_context = review[max(0, target_i - window):target_i]
        right_context = review[target_i+1:min(len(review), target_i + window+1)]
          
#         print('left:',left_context)
#         print('right:', right_context)
        
        layer_1 = np.mean(weight_0_1[left_context + right_context], axis=0)  # array (50,)
        #weight_1_2[target_samples].T).shape (50,6) 
        
        layer_2 = sigmoid(layer_1.dot(weight_1_2[target_samples].T))  #shape (6,)

        layer_2_delta = layer_2 - layer_2_target   #(6,0)
        layer_1_delta = layer_2_delta.dot(weight_1_2[target_samples]) #layer_1_delta.shape - (50,)
        
        weight_0_1[left_context+right_context] -= layer_1_delta*alpha
        weight_1_2[target_samples] -= np.outer(layer_2_delta, layer_1) * alpha
        
#     if (rev_i%250 == 0):
#         sys.stdout.write('\rProgress:' + str(rev_i/float(len(input_dataset*iterations))) 
#                                              + " " + str(similar('terrible')))
    #sys.stdout.write('\rProgress:' + str(rev_i/float(len(input_dataset)*iterations)))

    if (rev_i%2500 == 0):
        print(f'Progress: {(round(rev_i/float(len(input_dataset*iterations)), 3))*100}% \
              | Close meaning: {similar("terrible")}')

print(similar('terrible'))

Progress: 0.0%               | Close meaning: [('terrible', -0.0), ('jobmore', -0.3756935395892518), ('"lovely', -0.4001037601932979), ('brereton', -0.40698162747495154), ('"snap"', -0.40844424099482457), ('"hostel"', -0.4128899058173174), ('rudiments', -0.4182332242005236), ('"h', -0.4245184835125415), ('tonenealwelles', -0.42456282233050263), ('"blow', -0.42800789502114156)]
Progress: 5.0%               | Close meaning: [('terrible', -0.0), ('horrible', -1.560521218727666), ('brilliant', -1.855462515608804), ('poorly', -1.9271112456351245), ('decent', -1.937621159148777), ('solid', -1.980501889420204), ('porn', -1.999708293170252), ('disappointing', -2.001079215235697), ('simple', -2.034207861580273), ('remarkable', -2.044195328209629)]
Progress: 10.0%               | Close meaning: [('terrible', -0.0), ('cute', -2.6243340959369603), ('weak', -2.639840243343166), ('brilliant', -2.6569897673114795), ('cheesy', -2.7801319543829135), ('mediocre', -2.800403233941436), ('successful', -2.9

In [89]:
def analogy(positive = ['elizabeth', 'he'], negative = ['she']):
    
    #Сначала поэлементное умножение, потом сумма в строку
    norms = np.sum(weight_0_1 * weight_0_1, axis=1)
    #print(weight_0_1.shape)   - (123950, 50)
    #print(norms.shape)        - (123950,) 
    
    norms.resize(norms.shape[0], 1)
    print(norms.shape)

    normed_weights = weight_0_1 * norms
    
    #print(normed_weights.shape)  - (123950, 50)
    #print(len(weight_0_1[0]))   - 50
    
    query_vect = np.zeros(len(weight_0_1[0]))
    
    for word in positive:
        query_vect += normed_weights[word2index[word]]
    for word in negative:
        query_vect -= normed_weights[word2index[word]]
        
    scores = Counter()
    for word, index in word2index.items():
        raw_difference = weight_0_1[index] - query_vect
        squared_difference = raw_difference * raw_difference
        scores[word] = -math.sqrt(sum(squared_difference))
    return scores.most_common(10)[1:]


(123950, 1)
[('he', -899.8595842700796), ('nobody', -901.7350352288688), ('they', -902.1837026153111), ('i', -902.2831358086463), ('happen', -902.538423103598), ('husband', -902.6002194302539), ('it', -902.7367791213763), ('someone', -902.8475126272233), ('u', -902.90825458605)]
